In [11]:
import momepy
import geopandas as gpd
import matplotlib.pyplot as plt
from libpysal.weights import Queen, Rook, KNN
from shapely.ops import triangulate, transform
from shapely.geometry import MultiPoint
%matplotlib inline

In [2]:
df = gpd.read_file('parcel_pts.shp')

In [3]:
df = df.reset_index()[['index', 'geometry']]
df = df.rename(columns={'index': 'ID'})
df.head()

,ID,geometry
0,0,POINT (-105.22952 39.75626)
1,1,POINT (-105.22893 39.75609)
2,2,POINT (-105.22655 39.75729)
3,3,POINT (-105.22700 39.75781)
4,4,POINT (-105.22719 39.75756)


In [8]:
def ID_2geom(df):
        """ Transfers the ELEV value from the attribute table
        to the geometry's z-slot
        """
        df['geometry'] = df['geometry'].apply(
            lambda p: transform(
            lambda x, y: (x, y, df.loc[df['geometry'] == p, 'ID']), p)
            )
        
        return df[['ID', 'geometry']]

In [9]:
df = ID_2geom(df)

In [10]:
df

,ID,geometry
0,0,POINT Z (-105.22952 39.75626 0.00000)
1,1,POINT Z (-105.22893 39.75609 1.00000)
2,2,POINT Z (-105.22655 39.75729 2.00000)
3,3,POINT Z (-105.22700 39.75781 3.00000)
4,4,POINT Z (-105.22719 39.75756 4.00000)
...,...,...
81,81,POINT Z (-105.22816 39.75694 81.00000)
82,82,POINT Z (-105.22826 39.75688 82.00000)
83,83,POINT Z (-105.22846 39.75678 83.00000)
84,84,POINT Z (-105.22851 39.75675 84.00000)


In [12]:
df_mutltigeom = MultiPoint(df['geometry'].tolist())
tin = triangulate(df_mutltigeom)

In [15]:
df_tin = gpd.GeoDataFrame(geometry=tin, crs=4326)
df_tin.head()

,geometry
0,"POLYGON Z ((-105.22952 39.75626 0.00000, -105...."
1,"POLYGON Z ((-105.22952 39.75626 0.00000, -105...."
2,"POLYGON Z ((-105.22952 39.75626 0.00000, -105...."
3,"POLYGON Z ((-105.22816 39.75715 77.00000, -105..."
4,"POLYGON Z ((-105.22816 39.75715 77.00000, -105..."


In [16]:
def build_table_from_geometry(zTIN):
    """ Builds a the final attrib. table from the geometry
    column.
    """
    for index, row in zTIN.iterrows():
        idx = 0
        for pt in list(row['geometry'].exterior.coords):
            lat = pt[1]
            long = pt[0]
            z = pt[2]
            zTIN.loc[index, f'PT_{idx}_LAT'] = lat
            zTIN.loc[index, f'PT_{idx}_LONG'] = long
            zTIN.loc[index, f'PT_{idx}_ID'] = z
            idx+=1
    return zTIN

In [17]:
df_tin = build_table_from_geometry(df_tin)

In [18]:
df_tin.head()

,geometry,PT_0_LAT,PT_0_LONG,PT_0_ID,PT_1_LAT,PT_1_LONG,PT_1_ID,PT_2_LAT,PT_2_LONG,PT_2_ID,PT_3_LAT,PT_3_LONG,PT_3_ID
0,"POLYGON Z ((-105.22952 39.75626 0.00000, -105....",39.756262,-105.229519,0.0,39.755485,-105.228691,33.0,39.755945,-105.229118,32.0,39.756262,-105.229519,0.0
1,"POLYGON Z ((-105.22952 39.75626 0.00000, -105....",39.756262,-105.229519,0.0,39.755945,-105.229118,32.0,39.756280,-105.229488,50.0,39.756262,-105.229519,0.0
2,"POLYGON Z ((-105.22952 39.75626 0.00000, -105....",39.756262,-105.229519,0.0,39.756280,-105.229488,50.0,39.757151,-105.228165,77.0,39.756262,-105.229519,0.0
3,"POLYGON Z ((-105.22816 39.75715 77.00000, -105...",39.757151,-105.228165,77.0,39.756280,-105.229488,50.0,39.756341,-105.229380,51.0,39.757151,-105.228165,77.0
4,"POLYGON Z ((-105.22816 39.75715 77.00000, -105...",39.757151,-105.228165,77.0,39.756341,-105.229380,51.0,39.756371,-105.229325,52.0,39.757151,-105.228165,77.0


In [19]:
df_tin.to_file('tin.shp')